In [0]:
%sql
CREATE VIEW IF NOT EXISTS workspace.gold_staging.stg_methods
AS
SELECT
  time,
  methods.sop,
  methods.id AS method_id,
  methods.meth_id,
  methods.temp_id,
  methods.report_template_version,
  methods.seq_id,
  methods.seq_version
FROM workspace.silver_schema.silver_delta_table;
